In [1]:
# ====================================================================
# 08d_: Combined Age Band Cleaning and ICB Export Logic
# ====================================================================

import pandas as pd
from pathlib import Path

print("Starting Population Data Ingestion and ICB Age Segment Extraction...")


Starting Population Data Ingestion and ICB Age Segment Extraction...


In [3]:
# --------------------------------------------------------------------
# 1. Load and clean raw ONS LSOA-level population data
# --------------------------------------------------------------------
raw_data_file = Path("/Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/raw/ONS_2021_Census_Age_Segements_by_LSOA.csv")
processed_dir = Path("data/processed/")
processed_dir.mkdir(parents=True, exist_ok=True)

# Load with 5-line skip (ONS Census format)
df_pop = pd.read_csv(raw_data_file, header=5)
print(f"Loaded raw population file: {raw_data_file.name} — Shape: {df_pop.shape}")


Loaded raw population file: ONS_2021_Census_Age_Segements_by_LSOA.csv — Shape: (35672, 20)


In [8]:
# --------------------------------------------------------------------
# Rename and validate ONS age-band population columns
# --------------------------------------------------------------------

# Rename expected columns
expected_columns = [
    'lsoa_raw', 'total_population',
    'age_0_4', 'age_5_9', 'age_10_14', 'age_15_19',
    'age_20_24', 'age_25_29', 'age_30_34', 'age_35_39',
    'age_40_44', 'age_45_49', 'age_50_54', 'age_55_59',
    'age_60_64', 'age_65_69', 'age_70_74', 'age_75_79',
    'age_80_84', 'age_85_plus'
]

if len(df_pop.columns) >= len(expected_columns):
    df_pop.columns = expected_columns + list(df_pop.columns[len(expected_columns):])
else:
    raise ValueError("Unexpected number of columns in ONS data.")

# Extract clean LSOA code from string
df_pop['lsoa21cd'] = df_pop['lsoa_raw'].str.extract(r'(\bE\d{8}\b)', expand=False)
missing_lsoas = df_pop['lsoa21cd'].isna().sum()
if missing_lsoas > 0:
    print(f"Warning: {missing_lsoas} rows have missing LSOA codes.")
else:
    print("LSOA code extraction successful.")

# Validate totals by comparing against sum of age bands
age_cols = [
    'age_0_4', 'age_5_9', 'age_10_14', 'age_15_19',
    'age_20_24', 'age_25_29', 'age_30_34', 'age_35_39',
    'age_40_44', 'age_45_49', 'age_50_54', 'age_55_59',
    'age_60_64', 'age_65_69', 'age_70_74', 'age_75_79',
    'age_80_84', 'age_85_plus'
]

df_pop['calculated_total'] = df_pop[age_cols].sum(axis=1)
mismatch = (df_pop['total_population'] - df_pop['calculated_total']).abs().sum()

if mismatch == 0:
    print("Validation passed: total_population matches age band sum.")
else:
    print(f"Warning: Discrepancy in totals. Replacing with calculated values. Total absolute difference: {mismatch}")
    df_pop['total_population'] = df_pop['calculated_total']

# Preview cleaned output
print("\nCleaned ONS Age Segment Data (first 5 rows):")
print(df_pop[['lsoa21cd', 'total_population'] + age_cols].head())


Validation passed: total_population matches age band sum.

Cleaned ONS Age Segment Data (first 5 rows):
    lsoa21cd  total_population  age_0_4  age_5_9  age_10_14  age_15_19  \
0  E01011954              2284      166      169        163        129   
1  E01011969              1347       61       54         59         66   
2  E01011970              1066       41       42         35         44   
3  E01011971              1321       47       73         92         99   
4  E01033465              1954      102      152        184        136   

   age_20_24  age_25_29  age_30_34  age_35_39  age_40_44  age_45_49  \
0        144        153        171        138        131        127   
1         63         42         75         74         56         63   
2         53         65         56         50         39         62   
3        104         68         64         72         81        118   
4         82        122        110        164        200        156   

   age_50_54  age_55_59 

In [9]:
# --------------------------------------------------------------------
# Save cleaned ONS LSOA-level population file
# --------------------------------------------------------------------

# Define target save path
output_path = Path("/Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/processed/icb_split_enriched/lsoa_population.csv")

# Select and save cleaned DataFrame
df_pop_final = df_pop[['lsoa21cd', 'total_population'] + age_cols].copy()
df_pop_final.to_csv(output_path, index=False)

print(f"Saved cleaned age band population file to: {output_path}")


Saved cleaned age band population file to: /Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/processed/icb_split_enriched/lsoa_population.csv


In [10]:
# --------------------------------------------------------------------
# Step 3 – Merge Cleaned Population into Enriched ICB Files (7x)
# --------------------------------------------------------------------

# Define paths
icb_input_dir = Path("/Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/processed/icb_split_enriched")
pop_file = icb_input_dir / "lsoa_population.csv"
output_dir = icb_input_dir / "with_population"
output_dir.mkdir(exist_ok=True)

# Load cleaned population data
df_pop = pd.read_csv(pop_file)
print(f"Loaded population data: {df_pop.shape[0]} rows")



Loaded population data: 35672 rows


In [11]:
# Locate enriched ICB files
icb_files = list(icb_input_dir.glob("*_no_geom_enriched.csv"))
print(f"Located {len(icb_files)} ICB enriched files")



Located 12 ICB enriched files


In [12]:
# Loop and merge
for file in icb_files:
    df_icb = pd.read_csv(file)

    if 'lsoa21cd' not in df_icb.columns:
        print(f"Skipping {file.name}: Missing 'lsoa21cd'")
        continue

    icb_lsoas = set(df_icb['lsoa21cd'].unique())
    pop_lsoas = set(df_pop['lsoa21cd'].unique())
    missing = icb_lsoas - pop_lsoas

    print(f"{file.name}: {len(icb_lsoas) - len(missing)} matched LSOAs, {len(missing)} unmatched")

    # Merge with left join
    df_merged = df_icb.merge(df_pop, on='lsoa21cd', how='left')

    # Check for missing population entries
    missing_rows = df_merged['total_population'].isna().sum()
    if missing_rows > 0:
        print(f"Warning: {missing_rows} rows in {file.name} have missing population data.")

    # Save output
    merged_path = output_dir / f"{file.stem}_enriched_with_pop.csv"
    df_merged.to_csv(merged_path, index=False)
    print(f"Saved: {merged_path.name}")

    # Preview for visual check
    display(df_merged.head(2))

NHS_Devon_Integrated_Care_Board_no_geom_enriched.csv: 727 matched LSOAs, 0 unmatched
Saved: NHS_Devon_Integrated_Care_Board_no_geom_enriched_enriched_with_pop.csv


,LSOA21CD,LSOA21NM,IMD_Rank,LAD22NM,Urban_rural_flag,ICB23NM,lsoa21cd,lsoa21nm,msoa21cd,msoa21nm,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01020173,South Hams 012B,27399.0,South Hams,Rural,NHS Devon Integrated Care Board,E01020173,South Hams 012B,E02004200,South Hams 012,...,52,48,80,86,141,112,107,118,72,65
1,E01020172,South Hams 012A,19560.0,South Hams,Rural,NHS Devon Integrated Care Board,E01020172,South Hams 012A,E02004200,South Hams 012,...,99,100,142,156,172,145,178,130,81,73


NHS_Dorset_Integrated_Care_Board_no_geom_enriched.csv: 462 matched LSOAs, 0 unmatched
Saved: NHS_Dorset_Integrated_Care_Board_no_geom_enriched_enriched_with_pop.csv


,LSOA21CD,LSOA21NM,IMD_Rank,LAD22NM,Urban_rural_flag,ICB23NM,lsoa21cd,lsoa21nm,msoa21cd,msoa21nm,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01020484,Dorset 042G,18033.0,Dorset,Rural,NHS Dorset Integrated Care Board,E01020484,Dorset 042G,E02004268,Dorset 042,...,63,84,107,112,124,104,112,79,62,51
1,E01020481,Dorset 042D,21081.0,Dorset,Rural,NHS Dorset Integrated Care Board,E01020481,Dorset 042D,E02004268,Dorset 042,...,51,57,68,116,158,131,159,104,60,82


NHS_Bristol_North_Somerset_and_South_Gloucestershire_Integrated_Care_Board_no_geom_enriched.csv: 581 matched LSOAs, 0 unmatched
Saved: NHS_Bristol_North_Somerset_and_South_Gloucestershire_Integrated_Care_Board_no_geom_enriched_enriched_with_pop.csv


...

,LSOA21CD,LSOA21NM,IMD_Rank,LAD22NM,Urban_rural_flag,ICB23NM,lsoa21cd,lsoa21nm,msoa21cd,msoa21nm,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01032350,Wychavon 013D,19706.0,Wychavon,Rural,NHS Herefordshire and Worcestershire Integrate...,E01032350,Wychavon 013D,E02006760,Wychavon 013,...,72,75,90,95,83,90,88,61,52,37
1,E01014032,Herefordshire 023B,16442.0,"Herefordshire, County of",Rural,NHS Herefordshire and Worcestershire Integrate...,E01014032,Herefordshire 023B,E02002927,Herefordshire 023,...,81,77,104,147,146,137,134,96,64,60


Unassigned_ICB_no_geom_enriched.csv: 0 matched LSOAs, 2 unmatched
Saved: Unassigned_ICB_no_geom_enriched_enriched_with_pop.csv


,LSOA21CD,LSOA21NM,IMD_Rank,LAD22NM,Urban_rural_flag,ICB23NM,lsoa21cd,lsoa21nm,msoa21cd,msoa21nm,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,W01001595,Monmouthshire 007F,NaN,Monmouthshire,Rural,Unassigned_ICB,W01001595,Monmouthshire 007F,W02000342,Monmouthshire 007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,W01001599,Monmouthshire 004F,NaN,Monmouthshire,Urban,Unassigned_ICB,W01001599,Monmouthshire 004F,W02000339,Monmouthshire 004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NHS_Buckinghamshire_Oxfordshire_and_Berkshire_West_Integrated_Care_Board_no_geom_enriched.csv: 11 matched LSOAs, 0 unmatched
Saved: NHS_Buckinghamshire_Oxfordshire_and_Berkshire_West_Integrated_Care_Board_no_geom_enriched_enriched_with_pop.csv


,LSOA21CD,LSOA21NM,IMD_Rank,LAD22NM,Urban_rural_flag,ICB23NM,lsoa21cd,lsoa21nm,msoa21cd,msoa21nm,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01028715,Vale of White Horse 016F,25732.0,Vale of White Horse,Rural,"NHS Buckinghamshire, Oxfordshire and Berkshire...",E01028715,Vale of White Horse 016F,E02006886,Vale of White Horse 016,...,154,145,169,164,127,87,114,89,59,42
1,E01016301,West Berkshire 018B,19275.0,West Berkshire,Rural,"NHS Buckinghamshire, Oxfordshire and Berkshire...",E01016301,West Berkshire 018B,E02003384,West Berkshire 018,...,96,150,187,189,173,135,125,106,87,82


In [15]:
# --------------------------------------------------------------------
# Step – Combine all ICBs and create eight projection files
# --------------------------------------------------------------------

# --- paths ----------------------------------------------------------
src_dir = Path(
    "/Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand"
    "/data/processed/icb_split_enriched/with_population"
)
proj_root = Path(
    "/Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand"
    "/data/processed/projections"
)
proj_root.mkdir(parents=True, exist_ok=True)

# --- growth factors (relative to 2021 baseline) ---------------------
growth = {
    2021: 1.0000,   # baseline
    2022: 1.0027,
    2023: 1.0109,
    2024: 1.0176,
    2025: 1.0209,
    2026: 1.0350,
    2031: 1.0710,
    2036: 1.1070,
}

# --- read & concat all ICB files -----------------------------------
icb_files = list(src_dir.glob("*_enriched_with_pop.csv"))
if not icb_files:
    raise FileNotFoundError("No *_enriched_with_pop.csv files found in with_population/.")

df_all = pd.concat((pd.read_csv(f) for f in icb_files), ignore_index=True)
print(f"Combined {len(icb_files)} ICB files → {df_all.shape[0]} LSOA rows")

# --- restrict to required columns ----------------------------------
age_cols = [
    "age_0_4", "age_5_9", "age_10_14", "age_15_19",
    "age_20_24", "age_25_29", "age_30_34", "age_35_39",
    "age_40_44", "age_45_49", "age_50_54", "age_55_59",
    "age_60_64", "age_65_69", "age_70_74", "age_75_79",
    "age_80_84", "age_85_plus",
]
base_cols = ["lsoa21cd", "lsoa21nm", "ICB23NM", "total_population"]
keep_cols = base_cols + age_cols

missing = [c for c in keep_cols if c not in df_all.columns]
if missing:
    raise KeyError(f"Missing expected columns: {missing}")

df_all = df_all[keep_cols]

# --- scale and save per year ---------------------------------------
numeric_cols = ["total_population"] + age_cols

for year, factor in growth.items():
    df_year = df_all.copy()
    df_year[numeric_cols] = (df_year[numeric_cols] * factor).round(2)

    out_dir = proj_root / str(year)
    out_dir.mkdir(exist_ok=True)
    out_path = out_dir / f"all_icbs_{year}.csv"
    df_year.to_csv(out_path, index=False)

    print(f"Saved {out_path}  –  multiplier {factor}")
    display(df_year.head(2))   # quick visual check


Combined 12 ICB files → 3475 LSOA rows
Saved /Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/processed/projections/2021/all_icbs_2021.csv  –  multiplier 1.0


,lsoa21cd,lsoa21nm,ICB23NM,total_population,age_0_4,age_5_9,age_10_14,age_15_19,age_20_24,age_25_29,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1420.0,43.0,69.0,74.0,86.0,62.0,45.0,...,63.0,84.0,107.0,112.0,124.0,104.0,112.0,79.0,62.0,51.0
1,E01020481,Dorset 042D,NHS Dorset Integrated Care Board,1324.0,39.0,32.0,40.0,31.0,52.0,47.0,...,51.0,57.0,68.0,116.0,158.0,131.0,159.0,104.0,60.0,82.0


Saved /Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/processed/projections/2022/all_icbs_2022.csv  –  multiplier 1.0027


,lsoa21cd,lsoa21nm,ICB23NM,total_population,age_0_4,age_5_9,age_10_14,age_15_19,age_20_24,age_25_29,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1423.83,43.12,69.19,74.20,86.23,62.17,45.12,...,63.17,84.23,107.29,112.30,124.33,104.28,112.30,79.21,62.17,51.14
1,E01020481,Dorset 042D,NHS Dorset Integrated Care Board,1327.57,39.11,32.09,40.11,31.08,52.14,47.13,...,51.14,57.15,68.18,116.31,158.43,131.35,159.43,104.28,60.16,82.22


Saved /Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/processed/projections/2023/all_icbs_2023.csv  –  multiplier 1.0109


...

,lsoa21cd,lsoa21nm,ICB23NM,total_population,age_0_4,age_5_9,age_10_14,age_15_19,age_20_24,age_25_29,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1469.70,44.50,71.41,76.59,89.01,64.17,46.58,...,65.20,86.94,110.74,115.92,128.34,107.64,115.92,81.76,64.17,52.78
1,E01020481,Dorset 042D,NHS Dorset Integrated Care Board,1370.34,40.36,33.12,41.40,32.08,53.82,48.64,...,52.78,59.00,70.38,120.06,163.53,135.58,164.56,107.64,62.10,84.87


Saved /Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/processed/projections/2031/all_icbs_2031.csv  –  multiplier 1.071


,lsoa21cd,lsoa21nm,ICB23NM,total_population,age_0_4,age_5_9,age_10_14,age_15_19,age_20_24,age_25_29,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1520.82,46.05,73.90,79.25,92.11,66.40,48.20,...,67.47,89.96,114.60,119.95,132.80,111.38,119.95,84.61,66.40,54.62
1,E01020481,Dorset 042D,NHS Dorset Integrated Care Board,1418.00,41.77,34.27,42.84,33.20,55.69,50.34,...,54.62,61.05,72.83,124.24,169.22,140.30,170.29,111.38,64.26,87.82


Saved /Users/rosstaylor/Downloads/Research Project/Code Folder/Research Project - Geospatial Health Demand/data/processed/projections/2036/all_icbs_2036.csv  –  multiplier 1.107


,lsoa21cd,lsoa21nm,ICB23NM,total_population,age_0_4,age_5_9,age_10_14,age_15_19,age_20_24,age_25_29,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1571.94,47.60,76.38,81.92,95.20,68.63,49.82,...,69.74,92.99,118.45,123.98,137.27,115.13,123.98,87.45,68.63,56.46
1,E01020481,Dorset 042D,NHS Dorset Integrated Care Board,1465.67,43.17,35.42,44.28,34.32,57.56,52.03,...,56.46,63.10,75.28,128.41,174.91,145.02,176.01,115.13,66.42,90.77
